<a href="https://colab.research.google.com/github/smaliyu/AfriNLP/blob/main/hausa_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training  a Base Model for Hate Detection in Tweets

In this notebook, I attempt to train a model that detects hate speech in tweets.

In [ ]:
# mount drive to persist output
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# change working directory to drive
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [1]:
# clone the repo for access to the datasets and content
!git clone https://github.com/smaliyu/AfriNLP.git

Cloning into 'AfriNLP'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 49 (delta 19), reused 26 (delta 5), pack-reused 0
Receiving objects: 100% (49/49), 1.02 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
# change working directory to the root directory of repo
%cd AfriNLP

/content/AfriNLP


In [3]:
!pwd

/content/AfriNLP


In [4]:
# necessary imports

import torch
import pandas as pd
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder



# Custom dataset class to prepare dataset for the encoder model

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




In [5]:
# Load the dataset
data = pd.read_csv('/content/AfriNLP/datasets/hausa_updated.csv')

# Extract texts and labels
texts = data['tweet'].tolist()
labels = data['label'].tolist()

# Label Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)



In [6]:
# To see the mapping of encoded labels to original labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Hate': 0, 'Normal': 1, 'Offensive': 2}


In [7]:
# Tokenization
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
max_length = 128
# Split the data into train and test sets first
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, random_state=42, test_size=0.2)

# Now tokenize each set
def tokenize(texts):
    return tokenizer(texts, add_special_tokens=True, max_length=max_length, padding='max_length', return_attention_mask=True, truncation=True)

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

# Create Custom Dataset
train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model and Optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
model.to(device)
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Training Loss: {avg_train_loss}")

# Evaluation
model.eval()
predictions = []

with torch.inference_mode():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {accuracy}")


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5, Average Training Loss: 0.6788099062295607
Epoch 2/5, Average Training Loss: 0.5101105534000161
Epoch 3/5, Average Training Loss: 0.43294250728263534
Epoch 4/5, Average Training Loss: 0.37637466400355835
Epoch 5/5, Average Training Loss: 0.3185444668015856
Test Accuracy: 0.8060278207109737


In [ ]:
# Save only the model's state dictionary
torch.save(model.state_dict(), 'hausa_base_model_state_dict.pth')

